# TA Guidance: Week 4 Lab – Data Wrangling - From Business Question to Analysis

## 🎯 Lab Overview

**Duration**: 75 minutes  
**Focus**: Data manipulation, aggregation, and joining with business context  
**Key Skills**: pandas operations, business analysis, data wrangling workflows

### Learning Objectives
By the end of this lab, students will be able to:
1. **Apply data manipulation techniques** to answer real business questions
2. **Perform joins and aggregations** to combine datasets for analysis
3. **Translate business requirements** into analytical code
4. **Work systematically** through complex data analysis workflows

### Connection to Course Content
- **Tuesday Slides**: Data manipulation, joining, aggregation concepts
- **Readings 10-12**: Manipulating Data, Summarizing Data, Joining Data
- **Business Focus**: Real-world retail analytics scenarios

## 📚 Pre-Lab Preparation

### Setup and Technical Considerations
- **Data Loading**: Uses `completejourney_py` package - students may need to install
- **Common Issues**: 
  - Package installation problems
  - Understanding the business scenario context
  - Confusion about join keys (household_key vs household_id)
- **Time Management**: This lab has 12+ questions - students will NOT finish all during class

### Key Teaching Points
1. **Business Context Matters**: Every analysis serves a real business need
2. **Systematic Approach**: Break complex questions into steps
3. **Data Quality**: Always check for missing values and data issues
4. **Progressive Complexity**: Start simple, build to advanced joins and aggregations

### Grouping Strategy
- **Group Size**: 2-4 students
- **Roles**: Consider rotating who "drives" the coding
- **Pacing**: Allow groups to work at different speeds
- **Support**: Provide hints rather than complete solutions during class

## 🔧 Setup and Data Loading

In [ ]:
# If you don't have completejourney_py installed, run: pip install completejourney-py
from completejourney_py import get_data
import pandas as pd
import numpy as np

# Load datasets
cj_data = get_data()
transactions = cj_data['transactions']
products = cj_data['products']
demographics = cj_data['demographics']

# Quick preview
print("Transactions shape:", transactions.shape)
print("Products shape:", products.shape) 
print("Demographics shape:", demographics.shape)

# Show key columns
print("\nTransactions columns:", list(transactions.columns))
print("Products columns:", list(products.columns))
print("Demographics columns:", list(demographics.columns))

## 📋 Complete Solutions Guide

### ⚠️ Teaching Notes:
- **Use these solutions for preparation and support**
- **Don't show complete solutions during class** - provide hints and guidance
- **Focus on the analytical thinking process** rather than just the code
- **Emphasize business interpretation** of results

## Part 1 – Data Audit & Basic Exploration

### Q0: Data Audit
**Teaching Focus**: Understanding your dataset before analysis
**Common Issues**: Students skip this step - emphasize its importance
**Time Estimate**: 5-7 minutes

In [ ]:
# Q0 COMPLETE SOLUTION

# Total number of transactions
num_transactions = transactions.shape[0]
print(f"Total transactions: {num_transactions:,}")

# Date range of transactions
min_date = transactions['transaction_timestamp'].min()
max_date = transactions['transaction_timestamp'].max()
print(f"Date range: {min_date} to {max_date}")

# Number of unique households and products
num_households = demographics.shape[0]
num_products = products.shape[0]
print(f"Households with demographics: {num_households:,}")
print(f"Total products: {num_products:,}")

# Summary statistics for sales_value
min_sales = transactions['sales_value'].min()
max_sales = transactions['sales_value'].max()
mean_sales = transactions['sales_value'].mean()
print(f"Sales value - Min: ${min_sales:.2f}, Max: ${max_sales:.2f}, Mean: ${mean_sales:.2f}")

# Additional insights for teaching
print(f"\nUnique households in transactions: {transactions['household_id'].nunique():,}")
print(f"Unique products in transactions: {transactions['product_id'].nunique():,}")

**💡 Teaching Points for Q0:**
- This is a **massive retail dataset** - 2.5M+ transactions
- **Date range spans 2+ years** - includes seasonal patterns
- **Not all households** in demographics appear in transactions
- **Sales values can be negative** (returns/adjustments)

### Q1: Peak Sales Day Analysis
**Teaching Focus**: Date manipulation and groupby operations
**Common Issues**: 
- Forgetting `.dt.date` for date extraction
- Confusion about groupby syntax
**Time Estimate**: 6-8 minutes

In [ ]:
# Q1 COMPLETE SOLUTION

# Step 1: Create a date column from transaction_timestamp
transactions['date'] = transactions['transaction_timestamp'].dt.date

# Step 2: Group by date and sum sales_value
daily_sales = transactions.groupby(['date'])['sales_value'].sum()

# Step 3: Sort in descending order and get the top result
peak_sales_day = daily_sales.sort_values(ascending=False).head(1)

print("Day with highest total sales:")
print(peak_sales_day)
print(f"\nPeak sales amount: ${peak_sales_day.values[0]:,.2f}")

# Additional business insights for discussion
print("\nTop 5 sales days:")
top_5_days = daily_sales.sort_values(ascending=False).head(5)
for date, sales in top_5_days.items():
    print(f"{date}: ${sales:,.2f}")

**💡 Teaching Points for Q1:**
- **Business Impact**: This informs $2M+ in labor cost decisions
- **Date extraction**: `.dt.date` removes time component
- **Peak day patterns**: Often holiday-related (discuss seasonality)
- **Operations implications**: How would you staff this peak day?

### Q2: Top Departments by Sales
**Teaching Focus**: Joins and aggregation
**Common Issues**: 
- Join syntax confusion
- Forgetting to specify join columns
**Time Estimate**: 6-8 minutes

In [ ]:
# Q2 COMPLETE SOLUTION

# Step 1: Join transactions to products on product_id
merged_data = transactions.merge(products, on='product_id', how='inner')

# Step 2: Group by department and sum sales_value
dept_sales = merged_data.groupby('department')['sales_value'].sum()

# Step 3: Sort and display top 5 departments
top_5_depts = dept_sales.sort_values(ascending=False).head(5)

print("Top 5 departments by total sales:")
for dept, sales in top_5_depts.items():
    print(f"{dept}: ${sales:,.2f}")

# Additional insights
total_sales = dept_sales.sum()
print(f"\nTop department represents {top_5_depts.iloc[0]/total_sales:.1%} of total sales")

**💡 Teaching Points for Q2:**
- **Store management**: Which departments deserve prime floor space?
- **Inner join**: Only keeps products that exist in both datasets
- **Business insight**: Grocery typically dominates, but what's #2?
- **Strategic decisions**: How would this influence promotional budgets?

## Part 2 – Pricing & Data Quality Analysis

### Q3: Average Unit Price by Department
**Teaching Focus**: Creating calculated columns and handling data quality

**Common Issues**: 
- Division by zero (quantity = 0)
- Missing values in calculations

**Time Estimate**: 8-10 minutes

In [ ]:
# Q3 COMPLETE SOLUTION

# Step 1: Create unit_price column
transactions['unit_price'] = transactions['sales_value'] / transactions['quantity']

# Step 2: Join transactions to products to bring in department
merged_data = transactions.merge(products, on='product_id', how='inner')

# Step 3: Calculate average unit price by department
avg_unit_price = merged_data.groupby('department')['unit_price'].mean()

print("Average unit price by department:")
avg_unit_price.sort_values(ascending=False)

**💡 Teaching Points for Q3:**
- **Pricing strategy**: Which departments have premium positioning?
- **Data quality**: Division by zero creates infinite values
- **Business context**: How do these prices compare to competitors?
- **Calculated columns**: Creating new metrics from existing data

### Q4: Data Quality Investigation
**Teaching Focus**: Data quality assessment and troubleshooting

**Common Issues**: 
- Not understanding why missing values occur
- Forgetting to investigate root causes
  
**Time Estimate**: 5-7 minutes

In [ ]:
# Q4 COMPLETE SOLUTION

# Check for missing values in unit_price
missing_unit_prices = transactions['unit_price'].isna().sum()
print(f"Missing unit prices: {missing_unit_prices:,}")

# Check for infinite values (division by zero)
infinite_unit_prices = np.isinf(transactions['unit_price']).sum()
print(f"Infinite unit prices: {infinite_unit_prices:,}")

# Investigate the cause - quantity = 0
zero_quantity = (transactions['quantity'] == 0).sum()
print(f"Transactions with quantity = 0: {zero_quantity:,}")

# Show some examples
print("\nExamples of zero quantity transactions:")
zero_qty_examples = transactions[transactions['quantity'] == 0].head()
print(zero_qty_examples[['sales_value', 'quantity', 'unit_price']].head())

# Business interpretation
total_transactions = len(transactions)
percent_problematic = (zero_quantity / total_transactions) * 100
print(f"\nProblematic transactions: {percent_problematic:.2f}% of total")

**💡 Teaching Points for Q4:**
- **Finance impact**: Bad data affects financial reporting accuracy
- **Root cause analysis**: Always investigate WHY missing values occur
- **Business context**: Zero quantity might be returns or adjustments
- **Data cleaning**: How would you handle these cases in production?

## Part 3 – Customer Segmentation & Behavioral Analysis

### Q5: Income Level Spending Analysis

**Teaching Focus**: Joining demographic data and customer segmentation

**Common Issues**: 
- Confusion about join keys (household_key vs household_id)
- Not understanding income level categories

**Time Estimate**: 8-10 minutes

In [ ]:
# Q5 COMPLETE SOLUTION

# Step 1: Join transactions to demographics
# Note: Check the actual column names in your dataset
merged_data = transactions.merge(demographics, on='household_id', how='inner')

# Step 2-4: Group by income, calculate mean, and sort
income_spending = merged_data.groupby('income')['sales_value'].mean().sort_values(ascending=False)

print("Average spending by income level:")
print(income_spending)

# Additional business insights
print("\nCustomer count by income level:")
income_counts = merged_data.groupby('income')['household_id'].nunique().sort_values(ascending=False)
print(income_counts)

In [ ]:
# Marketing budget allocation insight
print("\nMarketing Priority Analysis:")
marketing_data = pd.DataFrame({
    'avg_spending': income_spending,
    'household_count': income_counts
})

marketing_data['total_potential'] = marketing_data['avg_spending'] * marketing_data['household_count']
print(marketing_data.sort_values('total_potential', ascending=False))

**💡 Teaching Points for Q5:**
- **Marketing strategy**: Higher income doesn't always mean higher spending per transaction
- **Customer segmentation**: Different income levels have different shopping patterns
- **Budget allocation**: Consider both spending level AND customer count
- **Join keys**: household_key connects transactions to demographics

### Q6: Family vs Non-Family Spending
**Teaching Focus**: Handling non-numeric categorical data
**Common Issues**: 
- kid_count is stored as strings, not numbers
- Understanding the fillna() and != logic
**Time Estimate**: 8-10 minutes

In [ ]:
# Q6 COMPLETE SOLUTION

# Step 1: Create has_kids variable (note that kid_count is not numeric!)
demographics['has_kids'] = demographics['kids_count'].fillna('0') != '0'

# Step 2: Join transactions to demographics and analyze spending by has_kids
merged_data = transactions.merge(demographics, on='household_id', how='inner')

# Calculate average spending by family status
family_spending = merged_data.groupby('has_kids')['sales_value'].agg(['mean', 'count', 'sum'])

print("Spending analysis: Families vs Non-Families")
print(family_spending)

# More detailed business analysis
families_avg = family_spending.loc[True, 'mean']
non_families_avg = family_spending.loc[False, 'mean']
difference = families_avg - non_families_avg
percent_diff = (difference / non_families_avg) * 100

print(f"\nBusiness Insight:")
print(f"Families spend ${families_avg:.2f} on average")
print(f"Non-families spend ${non_families_avg:.2f} on average")
print(f"Difference: ${difference:.2f} ({percent_diff:+.1f}%)")

if families_avg > non_families_avg:
    print("\n✅ RECOMMENDATION: Families spend more - expand family product lines!")
else:
    print("\n❌ FINDING: Non-families spend more - reconsider family expansion strategy")

**💡 Teaching Points for Q6:**
- **Data types matter**: kid_count is categorical, not numeric
- **Business validation**: Does this support family product line expansion?
- **Merchandising impact**: How should this influence product selection?
- **Boolean logic**: Understanding fillna() and string comparison

### Q7: Inventory Optimization Analysis
**Teaching Focus**: Volume vs revenue analysis
**Common Issues**: 
- Confusing quantity vs sales_value metrics
- Not seeing the supply chain implications
**Time Estimate**: 6-8 minutes

In [ ]:
# Q7 COMPLETE SOLUTION

# Join transactions to products to get department information
merged_data = transactions.merge(products, on='product_id', how='inner')

# Group by department and sum quantity
dept_quantity = merged_data.groupby('department')['quantity'].sum()

# Sort and display top 5
top_5_volume_depts = dept_quantity.sort_values(ascending=False).head(5)

print("Top 5 departments by total quantity sold:")
for dept, qty in top_5_volume_depts.items():
    print(f"{dept}: {qty:,.0f} items")

# Compare with sales dollars (from Q2)
print("\nSupply Chain Insight - Volume vs Revenue comparison:")
dept_sales = merged_data.groupby('department')['sales_value'].sum()
comparison = pd.DataFrame({
    'total_quantity': dept_quantity,
    'total_sales': dept_sales,
    'avg_price_per_item': dept_sales / dept_quantity
})

print("\nTop 5 by volume vs revenue:")
print(comparison.sort_values('total_quantity', ascending=False).head())

**💡 Teaching Points for Q7:**
- **Supply chain strategy**: High volume departments need different logistics
- **Storage optimization**: Volume matters more than revenue for space planning
- **Operational efficiency**: Which departments drive fulfillment complexity?
- **Volume vs value**: Some departments sell many low-value items

## Part 4 – Advanced Business Intelligence & Strategic Insights

### Q8: Procurement Priority Product

**Teaching Focus**: Product-level analysis and vendor negotiations

**Common Issues**: 
- Multi-step analysis (aggregate then join)
- Understanding procurement implications

**Time Estimate**: 8-10 minutes

In [ ]:
# Q8 COMPLETE SOLUTION

# Group by product_id and sum quantity, retain the top 10
product_volume = transactions.groupby('product_id', as_index=False)['quantity'].sum().sort_values(by='quantity', ascending=False).nlargest(10, 'quantity')
product_volume.merge(products, on='product_id', how='left')

**💡 Teaching Points for Q8:**
- **Procurement strategy**: High-volume items have the most negotiation leverage
- **Cost reduction**: Even small per-unit savings matter at high volumes
- **Vendor relationships**: These products represent key supplier partnerships
- **Multi-step analysis**: Aggregate first, then join for details

### Q9: Pizza Category Analysis
**Teaching Focus**: String filtering and category-specific analysis
**Common Issues**: 
- String filtering syntax
- Case sensitivity in searches
**Time Estimate**: 8-10 minutes

In [ ]:
# Q9 COMPLETE SOLUTION

# Filter products where product_type contains "pizza"
pizza_products = products[products['product_type'].str.contains('pizza', case=False, na=False)]

print(f"Found {len(pizza_products)} pizza products")

# Join pizza products to transactions
pizza_transactions = transactions.merge(pizza_products, on='product_id', how='inner')

# Group by product and sum sales_value
pizza_sales = pizza_transactions.groupby(['product_id', 'product_type'])['sales_value'].sum().sort_values(ascending=False)

print("\nTop 10 pizza products by total sales:")
print(pizza_sales.head(10))

# Category manager insights
total_pizza_sales = pizza_sales.sum()
top_pizza_sales = pizza_sales.iloc[0]
top_pizza_share = (top_pizza_sales / total_pizza_sales) * 100

print(f"\nCategory Manager Insights:")
print(f"Total pizza segment sales: ${total_pizza_sales:,.2f}")
print(f"Top pizza product represents {top_pizza_share:.1f}% of segment")
print(f"Top product: {pizza_sales.index[0][1]}")

**💡 Teaching Points for Q9:**
- **Category management**: Understanding segment performance within departments
- **Vendor negotiations**: Which pizza brands/products are most valuable?
- **String filtering**: `.str.contains()` with case=False for robust matching
- **Market concentration**: How concentrated is the pizza category?

### Q10: Premium Customer Segment Analysis

**Teaching Focus**: Multi-criteria filtering and premium market analysis

**Common Issues**: 
- Complex filtering logic
- Understanding demographic segments

**Time Estimate**: 10-12 minutes

In [ ]:
# Q10 COMPLETE SOLUTION

# First, identify the highest income level
print("Available income levels:")
print(demographics['income'].value_counts())

# Filter demographics for highest income level & has kids
highest_income = demographics['income'].max()  # This might need adjustment based on actual data
# Alternative approach - look for highest income descriptor
income_levels = demographics['income'].value_counts().index
print(f"\nUsing highest income level: {income_levels[0]}")

# Filter for highest income households with kids
premium_families = demographics[
    (demographics['income'] == income_levels[0]) & 
    (demographics['kids_count'].fillna('0') != '0')
]

print(f"\nPremium families (highest income + kids): {len(premium_families)} households")

# Join to transactions and products
premium_transactions = transactions.merge(premium_families, on='household_id', how='inner')
premium_full = premium_transactions.merge(products, on='product_id', how='inner')

# Group by category and sum sales_value
premium_category_sales = premium_full.groupby('product_category')['sales_value'].sum().sort_values(ascending=False)

print("\nTop categories for premium families:")
for category, sales in premium_category_sales.head(10).items():
    print(f"{category}: ${sales:,.2f}")

# Premium product buyer insights
total_premium_sales = premium_category_sales.sum()
top_category_share = (premium_category_sales.iloc[0] / total_premium_sales) * 100

print(f"\nProduct Buyer Insights:")
print(f"Top category represents {top_category_share:.1f}% of premium family spending")
print(f"Focus category for premium product selection: {premium_category_sales.index[0]}")

**💡 Teaching Points for Q10:**
- **Premium market strategy**: Understanding your most valuable customer segment
- **Product selection**: What categories drive revenue for high-value customers?
- **Complex filtering**: Multiple demographic criteria
- **Business focus**: Premium families represent key revenue opportunities

### Q11: Vendor Relationship Mapping
**Teaching Focus**: Manufacturer analysis and vendor management
**Common Issues**: 
- Two-step analysis (find top manufacturer, then analyze)
- Understanding vendor concentration
**Time Estimate**: 10-12 minutes

In [ ]:
# Q11 COMPLETE SOLUTION

# Join transactions to products to get manufacturer information
manufacturer_data = transactions.merge(products, on='product_id', how='inner')

# Group by manufacturer and sum sales_value
manufacturer_sales = manufacturer_data.groupby('manufacturer_id')['sales_value'].sum().sort_values(ascending=False)

print("Top 10 manufacturers by total sales:")
print(manufacturer_sales.head(10))

# Find the top manufacturer
top_manufacturer = manufacturer_sales.index[0]
top_manufacturer_sales = manufacturer_sales.iloc[0]

print(f"\nTop manufacturer: {top_manufacturer}")
print(f"Sales: ${top_manufacturer_sales:,.2f}")

# Find which departments this manufacturer primarily sells in
top_mfg_products = products[products['manufacturer_id'] == top_manufacturer]
dept_distribution = top_mfg_products['department'].value_counts()

print(f"\nDepartment distribution for {top_manufacturer}:")
for dept, count in dept_distribution.items():
    percent = (count / len(top_mfg_products)) * 100
    print(f"{dept}: {count} products ({percent:.1f}%)")

# Vendor management insights
total_sales = manufacturer_sales.sum()
top_vendor_share = (top_manufacturer_sales / total_sales) * 100

print(f"\nVendor Management Insights:")
print(f"Top vendor represents {top_vendor_share:.1f}% of total sales")
print(f"Primary department: {dept_distribution.index[0]}")
print(f"Contract negotiation priority: HIGH")

**💡 Teaching Points for Q11:**
- **Vendor concentration**: How dependent are you on key suppliers?
- **Contract negotiations**: Top vendors have significant leverage
- **Department focus**: Understanding where vendors compete
- **Risk management**: What happens if you lose this vendor?

### Q12: Customer Persona Development
**Teaching Focus**: Complex multi-table analysis and market segmentation
**Common Issues**: 
- Three-way joins
- Groupby with multiple columns
- Finding "top" category per income level
**Time Estimate**: 12-15 minutes

In [ ]:
# Q12 COMPLETE SOLUTION

# Join demographics → transactions → products
full_data = demographics.merge(transactions, on='household_id', how='inner')
full_data = full_data.merge(products, on='product_id', how='inner')

# Group by income level and category, sum quantity
income_category_qty = full_data.groupby(['income', 'product_category'])['quantity'].sum().reset_index()

# For each income level, find the top category
top_categories_by_income = income_category_qty.loc[income_category_qty.groupby('income')['quantity'].idxmax()]
print("Most frequently purchased category by income level:")
print(top_categories_by_income)


In [ ]:
# Additional persona insights
print("\nDetailed persona analysis:")
persona_analysis = full_data.groupby('income').agg({
    'household_id': 'nunique',
    'sales_value': ['sum', 'mean'],
    'quantity': 'sum'
}).round(2)

persona_analysis.columns = ['households', 'total_sales', 'avg_transaction', 'total_items']
print(persona_analysis)

# Marketing implications
print("\nMarketing Persona Recommendations:")
for _, row in top_categories_by_income.iterrows():
    income = row['income']
    category = row['product_category']
    print(f"• {income} segment: Focus advertising on {category} products")

**💡 Teaching Points for Q12:**
- **Customer segmentation**: Different income levels have different preferences
- **Marketing strategy**: Targeted advertising based on purchase behavior
- **Complex analysis**: Three-table joins with groupby operations
- **Business application**: How would marketing use these personas?

## 🎯 Lab Wrap-up and Assessment

### Key Concepts Students Should Master
By the end of this lab, students should demonstrate proficiency in:

1. **Data Manipulation Fundamentals**
   - Creating calculated columns (unit_price)
   - Date extraction and manipulation
   - Handling missing values and data quality issues

2. **Aggregation and Grouping**
   - Single-column groupby operations
   - Multiple aggregation functions (sum, mean, count)
   - Sorting and ranking results

3. **Joining Datasets**
   - Inner joins between transactions, products, and demographics
   - Understanding join keys and relationships
   - Multi-step analysis workflows

4. **Business Analysis Skills**
   - Translating business questions into analytical code
   - Interpreting results in business context
   - Understanding retail analytics applications

### Common Student Struggles and Solutions

| Issue | Symptom | Solution Approach |
|-------|---------|------------------|
| **Join Key Confusion** | Errors joining demographics | Emphasize household_key vs household_id |
| **Data Type Issues** | kid_count comparisons fail | Show string vs numeric data handling |
| **Missing Values** | Division by zero errors | Demonstrate data quality checks |
| **Complex Filtering** | Multi-criteria queries fail | Break into steps, explain boolean logic |
| **Business Context** | Focus only on code | Emphasize real-world implications |

### Reflection Questions
Use these to check student understanding:

1. **Technical Skills**: "Which join operation would you use if you wanted to keep all transactions, even for products not in the products table?"

2. **Business Application**: "How would the operations team actually use the peak sales day analysis in practice?"

3. **Data Quality**: "What other data quality issues might affect retail analytics?"

4. **Strategic Thinking**: "If you were the vendor management team, what additional analyses would you want?"

### Connection to Homework and Future Content
- **Homework Quiz**: Students use their analysis results to answer quiz questions
- **Next Week**: Building on joins and aggregations for more complex visualizations
- **Mid-term Project**: These retail analytics skills directly apply to the project

### Troubleshooting Guide

**If students finish early:**
- Explore additional business questions
- Calculate profitability metrics
- Analyze seasonal trends
- Help struggling classmates

**If students struggle with concepts:**
- Break complex questions into smaller steps
- Use business analogies to explain technical concepts
- Provide additional guided examples
- Focus on understanding over completion

**If technology issues arise:**
- Have backup datasets ready
- Know common import/installation fixes
- Pair students for screen sharing
- Prepare offline alternatives

---

## 📝 Post-Lab Notes for Instructor

**After each lab session, consider documenting:**
- Which questions took longer/shorter than expected
- Common student misconceptions that emerged
- Technical issues that arose
- Student engagement levels with business scenarios
- Suggestions for improving future iterations

This feedback helps refine the lab experience and ensure optimal learning outcomes for future classes.